In [1]:
import h5py
import numpy as np
import scanpy.api as sc

/cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
h5f = h5py.File("../simulated/test/6_groups/melanomaS2_sim_loc1.h5",'r')

In [3]:
h5f['matrix'].shape

(2495, 33694)

In [4]:
list(h5f['cell_attrs'].keys())

['cell_groups', 'cell_names', 'cells_on_rows']

In [5]:
cells_on_rows = h5f['cell_attrs/cells_on_rows']

In [6]:
h5f['gene_attrs']

<HDF5 group "/gene_attrs" (7 members)>

In [7]:
list(h5f['gene_attrs'].keys())

['DEFacGroup1',
 'DEFacGroup2',
 'DEFacGroup3',
 'DEFacGroup4',
 'DEFacGroup5',
 'DEFacGroup6',
 'gene_names']

### !Transpose the matrix
### Scanpy filtering functions assume that: genes=variables

In [8]:
matrix = h5f['matrix'].value

In [9]:
matrix.shape

(2495, 33694)

In [20]:
# mask 1
mask1 = (matrix.sum(axis=0) > 3)

In [21]:
filtered_matrix = matrix[:,matrix.sum(axis=0) > 3]

In [12]:
filtered_matrix.shape

(2495, 27294)

In [13]:
adata = sc.AnnData(filtered_matrix)

### Zheng17 Preprocessing

In [14]:
#sc.pp.filter_genes(adata, min_counts=1)  # only consider genes with more than 1 count
sc.pp.normalize_per_cell(adata)          # normalize with total UMI count per cell
filter_result = sc.pp.filter_genes_dispersion(adata.X, flavor='cell_ranger', n_top_genes=1000, log=False)

In [15]:
sum(filter_result.gene_subset)

1000

In [16]:
adata.shape

(2495, 27294)

In [17]:
adata = adata[:, filter_result.gene_subset]

In [18]:
adata.shape

(2495, 1000)

In [21]:
h5f['gene_attrs']['gene_names'].value

array([b'Gene1', b'Gene2', b'Gene3', ..., b'Gene33692', b'Gene33693',
       b'Gene33694'], dtype='|S10')

In [26]:
h5f['gene_attrs']['gene_names'].value[mask1][filter_result.gene_subset].shape

(1000,)

In [28]:
h5f.close()

## Testing